#  Example with Stanford SR830 Bundle

In [23]:
# IMPORTS

#%matplotlib notebook
import qcodes as qc
import os
import numpy as np
import time
#import progressbar
from time import sleep
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from SR830_ext import SR830_ext
from bundle_lockin import BundleLockin
from qcodes.instrument.base import Instrument
from qcodes.utils.validators import Numbers, Arrays
from qcodes.utils.dataset import doNd



from qcodes import initialise_or_create_database_at, load_or_create_experiment, load_by_id, Measurement
tutorial_db_path = os.path.join(os.getcwd(), 'tutorial_doND.db')
initialise_or_create_database_at(tutorial_db_path)
load_or_create_experiment(experiment_name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#1@C:\Users\Farzad\Documents\Rasmus\SR830ext\tutorial_doND.db
-----------------------------------------------------------------------------------
1-results-1-lockin_setpoints,lockin_trace-1
2-results-2-gen2_v_now,lockin_setpoints,lockin_trace-20
3-results-3-lockin_setpoints,lockin_trace-1
4-results-4-gen2_v_now,lockin_setpoints,lockin_trace-20
5-results-5-lockin_setpoints,lockin_trace-0
6-results-6-lockin_setpoints,lockin_trace-0
7-results-7-lockin_setpoints,lockin_trace-1
8-results-8-gen2_v_now,lockin_setpoints,lockin_trace-20
9-results-9-lockin_setpoints,lockin_trace-1
10-results-10-lockin_setpoints,lockin_trace-1
11-results-11-gen2_v_now,lockin_setpoints,lockin_trace-20
12-results-12-lockin_setpoints,lockin_trace-1
13-results-13-gen2_v_now,lockin_setpoints,lockin_trace-20

In [3]:
class DummyGenerator(Instrument):

    def __init__(self, name, **kwargs):

        super().__init__(name, **kwargs)

        self.add_parameter('v_start',
                           initial_value=0,
                           unit='V',
                           label='v start',
                           vals=Numbers(0,1e3),
                           get_cmd=None,
                           set_cmd=None)

        self.add_parameter('v_stop',
                           initial_value=1,
                           unit='V',
                           label='v stop',
                           vals=Numbers(1,1e3),
                           get_cmd=None,
                           set_cmd=None)
        
        self.add_parameter('v_now',
                           initial_value=0,
                           unit='V',
                           label='v_now',
                           vals=Numbers(self.v_start(),self.v_stop()),
                           get_cmd=None,
                           set_cmd=None)

In [4]:
slow = DummyGenerator('slow')
fast = DummyGenerator('fast')

In [5]:
# CONNECT TO INSTRUMENT

sr = SR830_ext('lockin', 'GPIB0::1::INSTR')

Connected to: Stanford_Research_Systems SR830 (serial:s/n40974, firmware:ver1.07) in 0.96s


In [6]:
sr.ch1_display('X')
sr.ch1_ratio('none')
sr.buffer_SR('Trigger')
sr.buffer_trig_mode.set('ON')
sr.set_sweep_parameters(fast.v_now, 0,0.5,20,0.2, label="Voltage")

In [7]:
lockins = (sr,)
bundle = BundleLockin('bundle',lockins)


In [8]:
bundle.set_sweep_parameters(fast.v_now, 0,0.5,20, label="Voltage")

In [9]:
bundle.setpoints.get()

array([0.        , 0.02631579, 0.05263158, 0.07894737, 0.10526316,
       0.13157895, 0.15789474, 0.18421053, 0.21052632, 0.23684211,
       0.26315789, 0.28947368, 0.31578947, 0.34210526, 0.36842105,
       0.39473684, 0.42105263, 0.44736842, 0.47368421, 0.5       ])

In [10]:
sr.buffer_reset()
for x in np.linspace(0,0.5,20):
    fast.v_now.set(x)
    sleep(0.1)
    sr.send_trigger()
    sr.ch1_databuffer.prepare_buffer_readout()    
 

In [11]:
bundle.trace_lockin.get()

array([-1.54785812e-06,  3.69362533e-06, -5.59445471e-06,  7.94604421e-06,
       -7.72345811e-06,  5.79189509e-06, -6.38235360e-06,  7.24568963e-07,
        7.30808824e-06, -3.56324017e-06,  5.08781523e-06, -7.16838986e-06,
        7.69179314e-06,  3.78582627e-06,  7.67782331e-06, -5.11016697e-06,
        8.45920295e-06, -7.46920705e-06,  6.83031976e-06, -5.05521894e-06])

In [12]:
bundle_parameters = bundle.__dict__['parameters']
traces = [bundle_parameters[key] for key in bundle_parameters.keys() if 'trace' in key]

In [13]:
lockins[0].name

'lockin'

In [35]:
meas = Measurement()
refresh_time = 1. # in s
delay_fast = 0.1
delay_slow = 0.1
num_points_fast = 20
num_points_slow = 20
start_slow = 0
end_slow = 0.5
start_fast = 0
end_fast = 0.5

bundle.set_sweep_parameters(fast.v_now, start_fast,end_fast,num_points_fast, label="Voltage")
interval_slow = np.linspace(start_slow,end_slow,num_points_slow)
meas.write_period = refresh_time
set_points_fast = bundle.setpoints

meas.register_parameter(set_points_fast)
fast.post_delay = delay_fast
 
meas.register_parameter(slow.v_now)
slow.post_delay = delay_slow


bundle_parameters = bundle.__dict__['parameters']
traces = [bundle_parameters[key] for key in bundle_parameters.keys() if 'trace' in key]
for trace in traces:
        meas.register_parameter(trace, setpoints=(slow.v_now, set_points_fast))

#progress_bar = progressbar.ProgressBar(max_value=num_points_slow * num_points_fast)
points_taken = 0
time.sleep(0.1)

with meas.run(write_in_background=True) as datasaver:
    run_id = datasaver.run_id
    last_time = time.time()
    for point_slow in interval_slow:
        slow.v_now.set(point_slow)
        
        for lockin in bundle.lockins:
            lockin.buffer_reset()
                
        for point_fast in set_points_fast.get():
            slow.v_now.set(point_fast)
            time.sleep(0.1)
            for lockin in bundle.lockins:
                lockin.send_trigger()
            points_taken += 1
            print(points_taken)
                    
        data = []
        for trace in traces:   
            data.append((trace,trace.get()))
        data.append((slow.v_now,slow.v_now.get()))
        data.append((set_points_fast,set_points_fast.get()))
        datasaver.add_result(*data)
        
 
    


Starting experimental run with id: 25. 
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267


## List of parameters

In [ ]:
doNd.do0d(sr.trace,do_plot=True)

In [ ]:
doNd.do1d(gen2.v_now, 0.01,1.0,20,1,sr.trace,do_plot=True)